# MNIST Multiclass Image Classification
We will train a neural network for hand-written digit recognition using the MNIST dataset consiting of digits 0-9. The MNIST dataset is taken from tensorflow datasets and is split into 70% for training the model and 30% for testing model accuracy.

In [1]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
import tensorflow_datasets as tfds

In [2]:
# load MNIST dataset from tensoflow datasets, 70% train set, 30% test set
(train_ds, test_ds), ds_info = tfds.load(
    'mnist',
    split=['train[:70%]', 'test[:30%]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
# prepare datasets
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

train_ds = train_ds.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(ds_info.splits['train'].num_examples)
train_ds = train_ds.batch(128)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

test_ds = test_ds.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(128)
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [4]:
# build and train model

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
  tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
  tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
  tf.keras.layers.Dense(10) # since there are 10 possible values for the digits (0-9)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # softmax classification
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    train_ds,
    epochs=11,
)

Epoch 1/11
329/329 [==============================] - 9s 10ms/step - loss: 0.7148 - sparse_categorical_accuracy: 0.9033
Epoch 2/11
329/329 [==============================] - 3s 9ms/step - loss: 0.4030 - sparse_categorical_accuracy: 0.9564
Epoch 3/11
329/329 [==============================] - 3s 9ms/step - loss: 0.3203 - sparse_categorical_accuracy: 0.9660
Epoch 4/11
329/329 [==============================] - 3s 10ms/step - loss: 0.2738 - sparse_categorical_accuracy: 0.9702
Epoch 5/11
329/329 [==============================] - 3s 9ms/step - loss: 0.2455 - sparse_categorical_accuracy: 0.9733
Epoch 6/11
329/329 [==============================] - 3s 10ms/step - loss: 0.2203 - sparse_categorical_accuracy: 0.9762
Epoch 7/11
329/329 [==============================] - 4s 11ms/step - loss: 0.2156 - sparse_categorical_accuracy: 0.9758
Epoch 8/11
329/329 [==============================] - 3s 8ms/step - loss: 0.1985 - sparse_categorical_accuracy: 0.9777
Epoch 9/11
329/329 [========================

In [5]:
# check accuracy on test set
model.evaluate(test_ds)

24/24 [==============================] - 2s 31ms/step - loss: 0.1865 - sparse_categorical_accuracy: 0.9723


[0.18652166426181793, 0.9723333120346069]